In [1]:
import os
import sys
import clr

from enum import Enum

In [2]:
clr.AddReference("System.Runtime.InteropServices")
from System.Runtime.InteropServices import Marshal

In [3]:
clr.AddReference(R'C:\Program Files\Computers and Structures\ETABS 21\ETABSv1.dll')
from ETABSv1 import *

In [4]:
helper = cHelper(Helper())

In [5]:
myETABSObject = cOAPI(helper.GetObject("CSI.ETABS.API.ETABSObject"))
ret=helper.GetObject('CSI.ETABS.API.ETABSObject')

In [6]:
SapModel=cSapModel(myETABSObject.SapModel)

In [7]:
ret=SapModel.SetPresentUnits(eUnits(14))

In [8]:
PropMaterial=cPropMaterial(SapModel.PropMaterial)
ret=PropMaterial.SetMaterial('Concrete',eMatType(2))
ret=PropMaterial.SetMPIsotropic('Concrete',15000*210**0.5,0.2,0.0000055)

In [9]:
ret=SapModel.SetPresentUnits(eUnits(8))

In [10]:
PropFrame=cPropFrame(SapModel.PropFrame)
ret=PropFrame.SetRectangle('B1','Concrete',0.15,0.15)

In [31]:
FrameObj=cFrameObj(SapModel.FrameObj)
F1=''
F2=''
F3=''
F4=''
F5=''
vld=0 # Volado 1m
tramo=2.6 # Espaciamiento entre vigas
[ret,F1]=FrameObj.AddByCoord(0,0,0,vld,0,0,F1,'B1')
[ret,F2]=FrameObj.AddByCoord(vld,0,0,vld+tramo,0,0,F2,'B1')
[ret,F3]=FrameObj.AddByCoord(vld+tramo,0,0,vld+2*tramo,0,0,F3,'B1')
[ret,F4]=FrameObj.AddByCoord(vld+2*tramo,0,0,vld+3*tramo,0,0,F4,'B1')
[ret,F5]=FrameObj.AddByCoord(vld+3*tramo,0,0,vld+3*tramo+vld,0,0,F5,'B1')

In [42]:
Analyze=cAnalyze(SapModel.Analyze)
ret=Analyze.RunAnalysis()

In [43]:
Results=cAnalysisResults(SapModel.Results)
Setup=cAnalysisResultsSetup(Results.Setup)
ret = Setup.DeselectAllCasesAndCombosForOutput()
ret = Setup.SetCaseSelectedForOutput('Dead')

In [45]:
ret=SapModel.SetPresentUnits(eUnits(12))
ProgramResult = [0,0,0,0,0,0,0,0,0,0,0,0]
NumberResults = 0
Obj = []
ObjSta = []
Elm = []
ElmSta = []
LoadCase = []
StepType = []
StepNum = []
P  = []
V2 = []
V3 = []
T  = []
M2 = []
M3 = []
[ret, NumberResults, Obj,ObjSta, Elm,ElmSta, LoadCase, StepType, StepNum, P ,V2,V3,T ,M2,M3] = Results.FrameForce('3',eItemTypeElm(0),NumberResults,Obj,ObjSta , Elm, ElmSta,LoadCase, StepType, StepNum, P ,V2,V3,T ,M2,M3)
for i in range(0,12):
    ProgramResult[i]=M3[i]
    print(round(ProgramResult[i],3))

-1.085
-0.215
0.654
1.524
2.394
2.829
2.829
2.195
0.927
-0.341
-1.609
-2.877


*Considerar 10 Output Stations para poder generar la data, automaticamente se asigna el orden asi que es bueno. Realizo no un trabajo muy sustancial que digamos*

In [ ]:
Momento Maximo
-0.0
1.063
2.125
3.188
3.678
3.678
3.099
2.024
0.949
-0.127
-1.202
-2.277

In [4]:
from sympy import *

In [5]:
B=symbols('B')

In [6]:
P=40+3.84*B**2+2*(0.64*B*(B-0.4)+0.08*(2*B-.8))
print (P)

3.84*B**2 + 1.28*B*(B - 0.4) + 0.32*B + 39.872


In [7]:
iq=1
iy=1
phi=33
gamma=1.6
Df=1.2
ex=6/P
ey=12/P
B_ef=B-2*ex
L_ef=2*B-2*ey
sy=1-0.2*B_ef/L_ef
Nq=exp(pi*tan(phi*pi/180))*tan((45+phi/2)*pi/180)**2
Ny=(Nq-1)*tan(1.4*phi*pi/180)
qd=(iq*gamma*Df*Nq+0.5*sy*iy*gamma*B*Ny)/3
qd.evalf(3)

8.72*B*(-0.8*(0.2*B - 2.4/(3.84*B**2 + 1.28*B*(B - 0.4) + 0.32*B + 39.9))/(2.0*B - 24.0/(3.84*B**2 + 1.28*B*(B - 0.4) + 0.32*B + 39.9)) + 0.8) + 16.7

In [ ]:
po=P/(2*B**2)
x=B/2
y=B
p=po*(1+3*ex*x/(B/2)**2+3*ey*y/(B**2))
solve(Eq(po*(1+3*ex*x/(B/2)**2+3*ey*y/(B**2)),(iq*gamma*Df*Nq+0.5*sy*iy*gamma*B*Ny)/3),B)